# Fanfooty Supercoach Prediction Model
Selecting your supercoach team is hard... this model is designed as a tool to assist supercoach users in selecting their team week-to-week.

This model uses rolling stats, opposition data, and past injury data to provide a predicted score for matches.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 30)

### Get Fanfooty Data
This data was scraped using scrape-fanfooty.pynb in this directory
- Remove bad SC data + convert set SC column as integer
- Create columns to allow merging with fixture data

In [2]:
df_fanfooty_player_raw = pd.read_csv("exports/scrape_20200718-184147/fanfooty_match_data_20200718-184147.csv", error_bad_lines=False, index_col=0)
df_fanfooty_player_raw = df_fanfooty_player_raw.loc[df_fanfooty_player_raw['SC'] != '-']
df_fanfooty_player_raw['SC'] = df_fanfooty_player_raw['SC'].astype('int64')
df_fanfooty_player_raw['match_identifier'] = df_fanfooty_player_raw['Fanfooty Match ID'].astype(str) + '_' + df_fanfooty_player_raw['Team']
df_fanfooty_player_raw['opposition_match_identifier'] = df_fanfooty_player_raw['Fanfooty Match ID'].astype(str) + '_' + df_fanfooty_player_raw['Opposition']
df_fanfooty_player_raw

C:\Users\richardm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\richardm\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,SC,null2,null3,null4,Kicks,Handballs,Marks,Tackles,Hitouts,Frees for,Frees against,Goals,Behinds,Not sure,Tag,Tag Notes,Tag 2,Tag 2 Notes,null5,null6,null7,null8,Position,Jumper Number,null9,null10,null11,DT own %,SC own %,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Opposition,match_identifier,opposition_match_identifier
0,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,990020.0,Andrew,Embley,WC,30,111,98,144,79,112,20,8,1,6,1,1,0,1,0,Full Time,gun,Dempsey going with him... %s from %O and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES
1,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,230254.0,Adam,Selwood,WC,50,107,107,143,79,108,10,9,4,11,0,3,2,1,0,Full Time,hot,Tagged by Lonergan... %D and %M with %T plus %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES
2,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,200112.0,Dean,Cox,WC,27,99,118,114,88,106,9,10,2,2,30,4,1,1,1,Full Time,news,%H and %P with %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES
3,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,240016.0,Beau,Waters,WC,26,98,84,130,79,117,15,13,5,6,0,0,4,0,0,Full Time,news,%P and %M with %F... clangers and FA dampening...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES
4,3425,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010,261911.0,Brad,Ebert,WC,26,94,109,121,70,96,12,9,3,6,0,1,0,1,0,Full Time,news,Matched up on Winderlich... %D and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90903,7354,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,291545.0,Mitch,Wallis,WB,13,37,58,43,30,42,1,6,1,4,0,0,1,1,0,Full Time,pocket,%P including %B... also %T... and scored %s......,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,3.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2.0,0.0,1.0,85.0,81.0,52.0,CA,7354_WB,7354_CA
90904,7354,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,996708.0,Bailey,Dale,WB,8,37,49,35,28,36,3,3,1,3,0,1,0,1,0,Full Time,spearhead,%D and %T plus %s... Starting deep forward on ...,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,31.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,0.0,66.0,79.0,23.0,CA,7354_WB,7354_CA
90905,7354,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,991939.0,Zaine,Cordy,WB,3,32,53,26,28,36,4,3,4,1,0,1,1,0,0,Full Time,job,%D and %M... In defence on McKay,NaN,NaN,NaN,NaN,NaN,NaN,Forward,12.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,1.0,85.0,100.0,148.0,CA,7354_WB,7354_CA
90906,7354,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020,291492.0,Josh,Bruce,WB,3,27,35,27,23,31,2,4,2,2,1,1,1,0,0,Full Time,spearhead,%O and %M plus %T... Up forward on Jones,NaN,NaN,NaN,NaN,NaN,NaN,Forward,17.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.0,0.0,2.0,100.0,84.0,108.0,CA,7354_WB,7354_CA


### Identify when players were injured during a match
Fanfooty has amazing "tags" that can be used to identify when a player has been injured during a match

If they have certain tags (e.g. concussed) and score below 80 supercoach points, they are judged as injured.

In [3]:
injured_tags = [
    'sore',
    'injured',
    'longterminjured',
    'concussed',
    'heart'
]

def get_injured_status(row):
    if (row['Tag'] in injured_tags or row['Tag 2'] in injured_tags) and row['SC'] < 80:
        return True
    else:
        return False

df_fanfooty_player_raw['Injured'] = df_fanfooty_player_raw.apply(lambda row: get_injured_status(row), axis=1)

### Get Fanfooty fixture data
This data was scraped using scrape-fanfooty.pynb in this directory

- Only pull in games that have SC total figures

In [4]:
df_fixture = pd.read_csv("exports/scrape_20200715-220533/fanfooty_fixture.csv", index_col=0)
df_fixture = df_fixture.loc[(df_fixture['Home SC total'] > 0 ) & (df_fixture['Away SC total'] > 0 )]
df_fixture

,FanFooty draw ID,year,competition,round,gametime (AET),day,home team,away team,ground,timeslot,TV coverage,home supergoals,home goals,home behinds,home points,away supergoals,away goals,away behinds,away points,match status,home_team_short,away_team_short,Home SC total,Away SC total
3191,3425,2010,HA,4,2010-04-16 20:40:00,Friday,West Coast,Essendon,Subiaco,N,Seven,0.0,15.0,11.0,101.0,0.0,11.0,12.0,78.0,Full Time,WC,ES,1739.0,1568.0
3192,3426,2010,HA,4,2010-04-17 14:10:00,Saturday,North Melbourne,Sydney,Docklands,D,Fox Sports,0.0,8.0,12.0,60.0,0.0,14.0,16.0,100.0,Full Time,NM,SY,1504.0,1797.0
3193,3427,2010,HA,4,2010-04-17 15:10:00,Saturday,Adelaide,Carlton,Footy Park,D,Ten,0.0,6.0,19.0,55.0,0.0,16.0,7.0,103.0,Full Time,AD,CA,1513.0,1826.0
3194,3428,2010,HA,4,2010-04-17 19:10:00,Saturday,Collingwood,Hawthorn,MCG,N,Ten,0.0,17.0,21.0,123.0,0.0,8.0,11.0,59.0,Full Time,CO,HW,1873.0,1447.0
3195,3429,2010,HA,4,2010-04-17 19:10:00,Saturday,Brisbane Lions,Western Bulldogs,Gabba,N,Fox Sports,0.0,13.0,23.0,101.0,0.0,12.0,7.0,79.0,Full Time,BL,WB,1781.0,1545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,7350,2020,HA,6,2020-07-11 18:05:00,Saturday,Melbourne,Gold Coast,Showground,N,NaN,0.0,12.0,8.0,80.0,0.0,9.0,9.0,63.0,Full Time,ME,GC,1641.0,1658.0
5560,7351,2020,HA,6,2020-07-11 19:40:00,Saturday,Essendon,North Melbourne,Gold Coast,N,NaN,0.0,9.0,13.0,67.0,0.0,7.0,11.0,53.0,Full Time,ES,NM,1812.0,1489.0
5561,7352,2020,HA,6,2020-07-12 13:05:00,Sunday,Port Adelaide,GWS,Gold Coast,D,NaN,0.0,9.0,9.0,63.0,0.0,6.0,10.0,46.0,Full Time,PA,WS,1657.0,1638.0
5562,7353,2020,HA,6,2020-07-12 15:35:00,Sunday,Richmond,Sydney,Gabba,T,NaN,0.0,4.0,10.0,34.0,0.0,3.0,8.0,26.0,Full Time,RI,SY,1699.0,1602.0


### Create a function that returns rolling averages for all stats given to it
e.g. I want the average supercoach score for a player for their last 5 games.

In [5]:
# Get rolling averages
def rolling_average(df, window):
    return df.rolling(min_periods=1, window=window).mean().shift(1)

### Get the opposition team's 5, 3, and 1 round rolling average Supercoach Score

- Split the fixture columns so there's a row for every team, for every game. As opposed to a single row for each game.

In [6]:
# Get rolling previous team SC scores
renamed_cols = {
    "home_team_short": "team", 
    "home points": "score",
    "Home SC total": "SC_total", 
    "away_team_short": "team", 
    "away points": "score", 
    "Away SC total": "SC_total", 
}

home_match_details = df_fixture.loc[:, ['year', 'gametime (AET)', 'ground', 'FanFooty draw ID', 'home_team_short', 'home points', 'Home SC total']].rename(columns=renamed_cols)
away_match_details = df_fixture.loc[:, ['year', 'gametime (AET)', 'ground', 'FanFooty draw ID', 'away_team_short', 'away points', 'Away SC total']].rename(columns=renamed_cols)
df_matches_by_team = pd.concat([home_match_details, away_match_details]).sort_index().reset_index()
df_matches_by_team['previous SC match score'] =  df_matches_by_team.groupby('team')['SC_total'].apply(lambda team: rolling_average(team, 1))
df_matches_by_team['rolling 3 round SC match average'] =  df_matches_by_team.groupby('team')['SC_total'].apply(lambda team: rolling_average(team, 3))
df_matches_by_team['rolling 5 round SC match average'] =  df_matches_by_team.groupby('team')['SC_total'].apply(lambda team: rolling_average(team, 5))
df_matches_by_team['match_identifier'] = df_matches_by_team['FanFooty draw ID'].astype(str) + '_' + df_matches_by_team['team']
df_matches_by_team

,index,year,gametime (AET),ground,FanFooty draw ID,team,score,SC_total,previous SC match score,rolling 3 round SC match average,rolling 5 round SC match average,match_identifier
0,3191,2010,2010-04-16 20:40:00,Subiaco,3425,WC,101.0,1739.0,NaN,NaN,NaN,3425_WC
1,3191,2010,2010-04-16 20:40:00,Subiaco,3425,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
2,3192,2010,2010-04-17 14:10:00,Docklands,3426,SY,100.0,1797.0,NaN,NaN,NaN,3426_SY
3,3192,2010,2010-04-17 14:10:00,Docklands,3426,NM,60.0,1504.0,NaN,NaN,NaN,3426_NM
4,3193,2010,2010-04-17 15:10:00,Footy Park,3427,AD,55.0,1513.0,NaN,NaN,NaN,3427_AD
...,...,...,...,...,...,...,...,...,...,...,...,...
4117,5561,2020,2020-07-12 13:05:00,Gold Coast,7352,PA,63.0,1657.0,1504.0,1683.666667,1782.0,7352_PA
4118,5562,2020,2020-07-12 15:35:00,Gabba,7353,SY,26.0,1602.0,1517.0,1585.333333,1644.2,7353_SY
4119,5562,2020,2020-07-12 15:35:00,Gabba,7353,RI,34.0,1699.0,1700.0,1593.666667,1631.4,7353_RI
4120,5563,2020,2020-07-12 18:45:00,Gold Coast,7354,CA,103.0,1706.0,1775.0,1685.666667,1600.2,7354_CA


In [7]:
cols_to_use = df_matches_by_team.columns.difference(df_fanfooty_player_raw.columns)
cols_to_use.tolist()

['FanFooty draw ID',
 'SC_total',
 'gametime (AET)',
 'ground',
 'index',
 'previous SC match score',
 'rolling 3 round SC match average',
 'rolling 5 round SC match average',
 'score',
 'team',
 'year']

### Feature Building: Merge team + opposition rolling SC scores back into Fanfooty dataset

In [8]:
df_fanfooty_player_raw = pd.merge(
    df_fanfooty_player_raw,
    df_matches_by_team,
    how='left',
    on='match_identifier', 
    suffixes=('_player', '_fixture')
)
df_fanfooty_player_raw

,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,SC,null2,null3,null4,Kicks,Handballs,Marks,Tackles,Hitouts,Frees for,Frees against,Goals,Behinds,Not sure,Tag,Tag Notes,Tag 2,Tag 2 Notes,null5,null6,null7,null8,Position,Jumper Number,null9,null10,null11,DT own %,SC own %,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Opposition,match_identifier,opposition_match_identifier,Injured,FanFooty draw ID,SC_total,gametime (AET),ground,index,previous SC match score,rolling 3 round SC match average,rolling 5 round SC match average,score,team,year
0,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,990020.0,Andrew,Embley,WC,30.0,111.0,98.0,144.0,79.0,112.0,20.0,8.0,1.0,6.0,1.0,1.0,0.0,1.0,0.0,Full Time,gun,Dempsey going with him... %s from %O and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3425.0,1739.0,2010-04-16 20:40:00,Subiaco,3191.0,NaN,NaN,NaN,101.0,WC,2010.0
1,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,230254.0,Adam,Selwood,WC,50.0,107.0,107.0,143.0,79.0,108.0,10.0,9.0,4.0,11.0,0.0,3.0,2.0,1.0,0.0,Full Time,hot,Tagged by Lonergan... %D and %M with %T plus %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3425.0,1568.0,2010-04-16 20:40:00,Subiaco,3191.0,NaN,NaN,NaN,78.0,ES,2010.0
2,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,200112.0,Dean,Cox,WC,27.0,99.0,118.0,114.0,88.0,106.0,9.0,10.0,2.0,2.0,30.0,4.0,1.0,1.0,1.0,Full Time,news,%H and %P with %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3426.0,1797.0,2010-04-17 14:10:00,Docklands,3192.0,NaN,NaN,NaN,100.0,SY,2010.0
3,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,240016.0,Beau,Waters,WC,26.0,98.0,84.0,130.0,79.0,117.0,15.0,13.0,5.0,6.0,0.0,0.0,4.0,0.0,0.0,Full Time,news,%P and %M with %F... clangers and FA dampening...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3426.0,1504.0,2010-04-17 14:10:00,Docklands,3192.0,NaN,NaN,NaN,60.0,NM,2010.0
4,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,261911.0,Brad,Ebert,WC,26.0,94.0,109.0,121.0,70.0,96.0,12.0,9.0,3.0,6.0,0.0,1.0,0.0,1.0,0.0,Full Time,news,Matched up on Winderlich... %D and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3427.0,1513.0,2010-04-17 15:10:00,Footy Park,3193.0,NaN,NaN,NaN,55.0,AD,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90903,7354.0,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020.0,291545.0,Mitch,Wallis,WB,13.0,37.0,58.0,43.0,30.0,42.0,1.0,6.0,1.0,4.0,0.0,0.0,1.0,1.0,0.0,Full Time,pocket,%P including %B... also %T... and scored %s......,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,3.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2.0,0.0,1.0,85.0,81.0,52.0,CA,7354_WB,7354_CA,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90904,7354.0,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020.0,996708.0,Bailey,Dale,WB,8.0,37.0,49.0,35.0,28.0,36.0,3.0,3.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,Full Time,spearhead,%D and %T plus %s... Starting deep forward on ...,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,31.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,0.0,66.0,79.0,23.0,CA,7354_WB,7354_CA,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90905,7354.0,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020.0,991939.0,Zaine,Cordy,WB,3.0,32.0,53.0,26.0,28.0,36.0,4.0,3.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,Full Time,job,%D and %M... In defence on McKay

In [9]:
df_fanfooty_player_raw = pd.merge(
    df_fanfooty_player_raw,
    df_matches_by_team,
    how='left',
    left_on='opposition_match_identifier', 
    right_on='match_identifier', 
#     suffixes=('_player', '_opposition')
)

In [10]:
df_fanfooty_player_raw

,Fanfooty Match ID,Fanfooty Match URL,Round,Year,Player ID,First Name,Surname,Team,null,DT,SC,null2,null3,null4,Kicks,Handballs,Marks,Tackles,Hitouts,Frees for,Frees against,Goals,Behinds,Not sure,Tag,Tag Notes,Tag 2,Tag 2 Notes,null5,null6,null7,null8,Position,Jumper Number,null9,null10,null11,DT own %,SC own %,AF own %,null12,AF Breakeven,null13,Contested Possessions,Clearances,Clangers,Disposal efficiency,Time on ground,Metres gained,Opposition,match_identifier_x,opposition_match_identifier,Injured,FanFooty draw ID_x,SC_total_x,gametime (AET)_x,ground_x,index_x,previous SC match score_x,rolling 3 round SC match average_x,rolling 5 round SC match average_x,score_x,team_x,year_x,index_y,year_y,gametime (AET)_y,ground_y,FanFooty draw ID_y,team_y,score_y,SC_total_y,previous SC match score_y,rolling 3 round SC match average_y,rolling 5 round SC match average_y,match_identifier_y
0,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,990020.0,Andrew,Embley,WC,30.0,111.0,98.0,144.0,79.0,112.0,20.0,8.0,1.0,6.0,1.0,1.0,0.0,1.0,0.0,Full Time,gun,Dempsey going with him... %s from %O and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3425.0,1739.0,2010-04-16 20:40:00,Subiaco,3191.0,NaN,NaN,NaN,101.0,WC,2010.0,3191.0,2010.0,2010-04-16 20:40:00,Subiaco,3425.0,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
1,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,230254.0,Adam,Selwood,WC,50.0,107.0,107.0,143.0,79.0,108.0,10.0,9.0,4.0,11.0,0.0,3.0,2.0,1.0,0.0,Full Time,hot,Tagged by Lonergan... %D and %M with %T plus %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3425.0,1568.0,2010-04-16 20:40:00,Subiaco,3191.0,NaN,NaN,NaN,78.0,ES,2010.0,3191.0,2010.0,2010-04-16 20:40:00,Subiaco,3425.0,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
2,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,200112.0,Dean,Cox,WC,27.0,99.0,118.0,114.0,88.0,106.0,9.0,10.0,2.0,2.0,30.0,4.0,1.0,1.0,1.0,Full Time,news,%H and %P with %s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3426.0,1797.0,2010-04-17 14:10:00,Docklands,3192.0,NaN,NaN,NaN,100.0,SY,2010.0,3191.0,2010.0,2010-04-16 20:40:00,Subiaco,3425.0,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
3,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,240016.0,Beau,Waters,WC,26.0,98.0,84.0,130.0,79.0,117.0,15.0,13.0,5.0,6.0,0.0,0.0,4.0,0.0,0.0,Full Time,news,%P and %M with %F... clangers and FA dampening...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3426.0,1504.0,2010-04-17 14:10:00,Docklands,3192.0,NaN,NaN,NaN,60.0,NM,2010.0,3191.0,2010.0,2010-04-16 20:40:00,Subiaco,3425.0,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
4,3425.0,http://live.fanfooty.com.au/game/matchcentre.h...,R4,2010.0,261911.0,Brad,Ebert,WC,26.0,94.0,109.0,121.0,70.0,96.0,12.0,9.0,3.0,6.0,0.0,1.0,0.0,1.0,0.0,Full Time,news,Matched up on Winderlich... %D and %T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ES,3425_WC,3425_ES,False,3427.0,1513.0,2010-04-17 15:10:00,Footy Park,3193.0,NaN,NaN,NaN,55.0,AD,2010.0,3191.0,2010.0,2010-04-16 20:40:00,Subiaco,3425.0,ES,78.0,1568.0,NaN,NaN,NaN,3425_ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90778,7354.0,http://live.fanfooty.com.au/game/matchcentre.h...,R6,2020.0,291545.0,Mitch,Wallis,WB,13.0,37.0,58.0,43.0,30.0,42.0,1.0,6.0,1.0,4.0,0.0,0.0,1.0,1.0,0.0,Full Time,pocket,%P including %B... also %T... and scored %s......,NaN,NaN,NaN,NaN,NaN,NaN,Midfielder,3.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2.0,0.0,1.0,85.